<a href="https://colab.research.google.com/github/Cempressa/modelagemEcommerce/blob/main/C%C3%B3digo_da_Aplica%C3%A7%C3%A3o_Flask_(app_py)_Simplificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importa as partes que precisamos do Flask (para criar a página web)
from flask import Flask, render_template, request, redirect, url_for

# Importa a ferramenta para conectar com o banco de dados MySQL
import mysql.connector

# Cria o nosso aplicativo Flask. É como dar um nome para o nosso programa web.
app = Flask(__name__)

# --- CONFIGURAÇÕES DO SEU BANCO DE DADOS MySQL ---
# ESTA É A PARTE MAIS IMPORTANTE E QUE VOCÊ PRECISA MUDAR!
# Se você digitou errado aqui, o programa não vai conseguir se conectar ao seu banco.
db_config = {
    'host': 'localhost',      # Geralmente 'localhost' se o MySQL está no seu próprio computador
    'user': 'SEU_USUARIO_MYSQL',    # EX: 'root'. Substitua pelo seu nome de usuário do MySQL!
    'password': 'SUA_SENHA_MYSQL',  # EX: 'minhasenha123'. Substitua pela sua senha do MySQL!
    'database': 'ordem_servico_db' # O nome do banco de dados que você criou com o código SQL anterior
}

# --- FUNÇÃO PARA CONECTAR AO BANCO DE DADOS ---
# Esta função é como a "ponte" entre nosso programa Python e o MySQL.
def get_db_connection():
    try:
        # Tenta fazer a conexão usando as configurações que você colocou acima
        conn = mysql.connector.connect(**db_config)
        return conn # Se conseguir, retorna a conexão
    except mysql.connector.Error as err:
        # Se der erro (ex: usuário ou senha errados, banco de dados não existe)
        print(f"ERRO AO CONECTAR AO BANCO DE DADOS: {err}")
        print("VERIFIQUE SEU USUÁRIO, SENHA E O NOME DO BANCO DE DADOS EM 'db_config' no código.")
        return None # Retorna "nada" se a conexão falhar

# --- ROTA PRINCIPAL (QUANDO VOCÊ ABRE O SITE) ---
# Quando você digita o endereço do seu site (ex: http://127.0.0.1:5000/)
# esta função é chamada.
@app.route('/')
def index():
    # Ela simplesmente redireciona você para a página de clientes.
    return redirect(url_for('listar_clientes'))

# --- ROTA PARA LISTAR CLIENTES (A PÁGINA QUE VOCÊ COLOu ANTES) ---
# Quando você vai para o endereço /clientes (ex: http://127.0.0.1:5000/clientes)
# esta função é executada.
@app.route('/clientes')
def listar_clientes():
    conn = get_db_connection() # Tenta se conectar ao banco de dados
    if conn is None: # Se a conexão falhou
        return "Desculpe, não conseguimos conectar ao banco de dados. Verifique as configurações no 'app.py'.", 500

    # Cria um "cursor" para enviar comandos SQL para o banco.
    # dictionary=True faz com que os resultados venham como "dicionários",
    # o que facilita pegar os dados pelo nome da coluna (ex: cliente['nome'])
    cursor = conn.cursor(dictionary=True)
    try:
        # Manda o comando SQL para selecionar todos os dados da tabela 'Clientes'
        cursor.execute("SELECT * FROM Clientes")
        clientes = cursor.fetchall() # Pega todos os resultados que o banco devolveu
    except mysql.connector.Error as err:
        print(f"ERRO AO EXECUTAR CONSULTA PARA LISTAR CLIENTES: {err}")
        clientes = [] # Se der erro na consulta, a lista de clientes fica vazia
    finally:
        # MUITO IMPORTANTE: Sempre feche o cursor e a conexão para liberar recursos do banco!
        cursor.close()
        conn.close()

    # Pede para o Flask mostrar o arquivo HTML 'lista_clientes.html'
    # e envia a lista de clientes (que pode estar vazia ou cheia) para ele.
    return render_template('lista_clientes.html', clientes=clientes)

# --- ROTA PARA ADICIONAR UM NOVO CLIENTE ---
# Esta rota serve para duas coisas:
# 1. Mostrar o formulário de cadastro (quando você acessa /clientes/novo com GET)
# 2. Receber os dados do formulário e salvar no banco (quando você envia o formulário com POST)
@app.route('/clientes/novo', methods=['GET', 'POST'])
def novo_cliente():
    if request.method == 'POST': # Se o formulário foi enviado (método POST)
        # Pega os dados que o usuário digitou no formulário
        nome = request.form['nome']
        # .get() é usado para campos que podem ser vazios/não preenchidos
        endereco = request.form.get('endereco')
        bairro = request.form.get('bairro')
        cidade = request.form.get('cidade')
        telefone = request.form.get('telefone')
        cpf_cnpj = request.form['cpf_cnpj'] # Este campo é obrigatório
        rg_ie = request.form.get('rg_ie')

        conn = get_db_connection() # Conecta ao banco de dados
        if conn is None:
            return "Desculpe, não conseguimos conectar ao banco de dados.", 500

        cursor = conn.cursor()
        try:
            # Comando SQL para inserir os dados na tabela 'Clientes'
            sql = """
                INSERT INTO Clientes (nome, endereco, bairro, cidade, telefone, cpf_cnpj, rg_ie)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            # Executa o comando SQL, passando os valores na ordem correta
            cursor.execute(sql, (nome, endereco, bairro, cidade, telefone, cpf_cnpj, rg_ie))
            conn.commit() # Confirma que as mudanças devem ser salvas no banco
            return redirect(url_for('listar_clientes')) # Redireciona de volta para a lista de clientes
        except mysql.connector.Error as err:
            print(f"ERRO AO INSERIR CLIENTE: {err}")
            conn.rollback() # Se der erro, desfaz a operação para não deixar dados "pela metade"
            # Em um sistema real, você mostraria uma mensagem de erro mais amigável ao usuário
            return "Erro ao cadastrar cliente. Pode ser um CPF/CNPJ duplicado ou outro problema.", 500
        finally:
            cursor.close()
            conn.close()
    # Se a requisição não for POST (ou seja, for GET), significa que
    # o usuário só quer ver o formulário de cadastro.
    return render_template('cadastro_cliente.html') # Mostra o formulário

# --- INICIA O APLICATIVO FLASK ---
# Esta parte é importante para que o programa comece a funcionar
if __name__ == '__main__':
    # 'debug=True' é ótimo para quando estamos desenvolvendo.
    # Ele mostra erros detalhados e reinicia o servidor automaticamente quando você muda o código.
    # Em um sistema real (que vai para o público), isso deve ser 'False' por segurança.
    app.run(debug=True)